## Load Necessary Library

In [40]:
from sklearn.metrics import accuracy_score,mean_absolute_error
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsClassifier,KNeighborsRegressor
from sklearn.pipeline import make_pipeline,make_union
from sklearn.preprocessing import MinMaxScaler, Imputer,PolynomialFeatures, StandardScaler
from sklearn.model_selection import GridSearchCV,KFold,train_test_split
import warnings
from sklearn.feature_selection import SelectPercentile,SelectKBest, VarianceThreshold
warnings.filterwarnings('ignore')
import numpy as np
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report
from sklearn.neural_network import MLPClassifier

In [2]:
import matplotlib.pyplot as plt
import pandas as pd

In [3]:
#Need to run all Data Prep Notebook cells
%store -r data

In [4]:
data.head()

,name,category,main_category,currency,deadline,goal,launched,pledged,state,backers,country,usd pledged,usd_pledged_real,usd_goal_real
ID,,,,,,,,,,,,,,
1000002330,The Songs of Adelaide & Abullah,108,12,5,2015-10-09,1000.0,2015-08-11 12:12:28,0.0,False,0,9,0.0,0.0,1533.95
1000003930,Greeting From Earth: ZGAC Arts Capsule For ET,93,6,13,2017-11-01,30000.0,2017-09-02 04:43:57,2421.0,False,15,22,100.0,2421.0,30000.00
1000004038,Where is Hank?,93,6,13,2013-02-26,45000.0,2013-01-12 00:20:50,220.0,False,3,22,220.0,220.0,45000.00
1000007540,ToshiCapital Rekordz Needs Help to Complete Album,90,10,13,2012-04-16,5000.0,2012-03-17 03:24:11,1.0,False,1,22,1.0,1.0,5000.00
1000011046,Community Film Project: The Art of Neighborhoo...,55,6,13,2015-08-29,19500.0,2015-07-04 08:35:03,1283.0,False,14,22,1283.0,1283.0,19500.00


## Make a new feature

In [5]:
data["duration"] = (data["deadline"]  - data['launched']).astype(int)

In [6]:
data.columns

Index(['name', 'category', 'main_category', 'currency', 'deadline', 'goal',
       'launched', 'pledged', 'state', 'backers', 'country', 'usd pledged',
       'usd_pledged_real', 'usd_goal_real', 'duration'],
      dtype='object')

In [7]:
feature = data[['category', 'main_category', 'currency', 'goal','duration', 'country']]
outcome = data[['state']]
feature_name = data[["name"]]

In [8]:
feature.head()

,category,main_category,currency,goal,duration,country
ID,,,,,,
1000002330,108,12,5,1000.0,5053652000000000,9
1000003930,93,6,13,30000.0,5166963000000000,22
1000004038,93,6,13,45000.0,3886750000000000,22
1000007540,90,10,13,5000.0,2579749000000000,22
1000011046,55,6,13,19500.0,4807497000000000,22


## Split test and train data set

In [9]:
train_features_small, validation_features, train_outcome_small, validation_outcome = train_test_split(
    feature,
    outcome,
    test_size = 0.2
)
train_name_small, validation_name_features, train_outcome_small_name, validation_name_outcome = train_test_split(
    feature_name,
    outcome,
    test_size = 0.2
)

# Predict the State by Its Name

In [10]:
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import SGDClassifier

In [11]:
text_clf = make_pipeline(
                    CountVectorizer(),
                    TfidfTransformer(),
                    SGDClassifier(penalty='l2', tol=None,max_iter=10)
)

param = {'countvectorizer__ngram_range': [(1, 1), (1, 2)],
    'tfidftransformer__use_idf': (True, False),
    "sgdclassifier__alpha":[1e-3,0.001],
}

grid_search_text = GridSearchCV(text_clf, n_jobs=-1,cv=5,iid=False,param_grid=param)
grid_search_text.fit(train_name_small.name,train_outcome_small_name)

GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=Pipeline(memory=None,
     steps=[('countvectorizer', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
  ...m_state=None, shuffle=True, tol=None,
       validation_fraction=0.1, verbose=0, warm_start=False))]),
       fit_params=None, iid=False, n_jobs=-1,
       param_grid={'countvectorizer__ngram_range': [(1, 1), (1, 2)], 'tfidftransformer__use_idf': (True, False), 'sgdclassifier__alpha': [0.001, 0.001]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [ ]:
print(classification_report(validation_outcome, grid_search_text.best_estimator_.predict(validation_name_features.name)))

In [49]:
text_clf = make_pipeline(
                    CountVectorizer(),
                    TfidfTransformer(),
                    SelectKBest(k=10),
                    MLPClassifier(hidden_layer_sizes=(500,10,2),  
                                  activation='relu', 
                                   batch_size='auto', 
                                  learning_rate_init=0.01, power_t=0.5, 
                                  max_iter=1000, shuffle=True,
                                  tol=0.0001, verbose=False, warm_start=False,
                                  momentum=0.9, nesterovs_momentum=True,
                                    early_stopping=False, validation_fraction=0.1,
                                  beta_1=0.9,beta_2=0.999, epsilon=1e-08)
                                )


param = {'countvectorizer__ngram_range': [(1, 1), (1, 2)],
    'tfidftransformer__use_idf': (True, False)
}

grid_search_text = GridSearchCV(text_clf, n_jobs=-1,cv=5,iid=False,param_grid=param)
grid_search_text.fit(train_name_small.name,train_outcome_small_name)

KeyboardInterrupt: 

In [47]:
print(classification_report(validation_outcome, grid_search_text.predict(validation_name_features.name)))


              precision    recall  f1-score   support

       False       0.65      1.00      0.78     48504
        True       0.00      0.00      0.00     26669

   micro avg       0.65      0.65      0.65     75173
   macro avg       0.32      0.50      0.39     75173
weighted avg       0.42      0.65      0.51     75173



In [48]:
grid_search_text.best_score_

AttributeError: 'GridSearchCV' object has no attribute 'best_score_'

In [42]:
grid_search_text.score(validation_name_features.name,validation_name_outcome)

0.642597741210275

# Predict the State by other value

In [19]:
from sklearn.ensemble import RandomForestClassifier
pipe = make_pipeline(VarianceThreshold(0.1),
                     SelectKBest(k=5),
                     RandomForestClassifier(max_features="auto",max_depth=4, min_samples_split=2)
                     )

params ={'randomforestclassifier__n_estimators':range(10,50,10)
        # 'selectkbest__k':range(1,7)
        }

grid_search_rf = GridSearchCV(pipe, param_grid = params,
                               cv = KFold(n_splits=10, shuffle=True),n_jobs = -1)

In [22]:
grid_search_rf.fit(train_features_small,train_outcome_small)

GridSearchCV(cv=KFold(n_splits=10, random_state=None, shuffle=True),
       error_score='raise-deprecating',
       estimator=Pipeline(memory=None,
     steps=[('variancethreshold', VarianceThreshold(threshold=0.1)), ('selectkbest', SelectKBest(k=5, score_func=<function f_classif at 0x11b02d950>)), ('randomforestclassifier', RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=4, max_features='auto', max_...obs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False))]),
       fit_params=None, iid='warn', n_jobs=-1,
       param_grid={'randomforestclassifier__n_estimators': range(10, 50, 10)},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [28]:
print(classification_report(validation_outcome, grid_search_rf.best_estimator_.predict(validation_features)))

              precision    recall  f1-score   support

       False       0.67      0.94      0.78     48504
        True       0.59      0.17      0.26     26669

   micro avg       0.66      0.66      0.66     75173
   macro avg       0.63      0.55      0.52     75173
weighted avg       0.64      0.66      0.60     75173



In [23]:
grid_search_rf.score(validation_features,validation_outcome)

0.6632833597169197

In [24]:
rf_proba = pipe.predict_log_proba(validation_features)

# Combine two Prediction Together

In [107]:
total_proba = name_proba * rf_proba

In [108]:
total_proba

array([[0.27718412, 0.79689983],
       [0.52655006, 0.43774873],
       [0.29568657, 0.75431854],
       ...,
       [0.10719979, 1.48391614],
       [0.657058  , 0.30049206],
       [0.4828025 , 0.47806432]])

In [112]:
total_pred = [(x[0] >= x[1])for x in total_proba]

In [113]:
validation_outcome["pred"] = total_pred

In [114]:
len(validation_outcome[validation_outcome["pred"]==validation_outcome["state"]])/len(validation_outcome)

0.643489018663616

### Neural Network test

In [116]:
from sklearn.neural_network import MLPClassifier

    
pipe = make_pipeline(VarianceThreshold(0.1),
                    SelectKBest(k = 10),
                     MLPClassifier(hidden_layer_sizes=(13,5,),  
                                  activation='relu', 
                                   batch_size='auto', 
                                  learning_rate_init=0.01, power_t=0.5, 
                                  max_iter=1000, shuffle=True,
                                  tol=0.0001, verbose=False, warm_start=False,
                                  momentum=0.9, nesterovs_momentum=True,
                                    early_stopping=False, validation_fraction=0.1,
                                  beta_1=0.9,beta_2=0.999, epsilon=1e-08)
                                )
params ={'mlpclassifier__alpha':[0.0001,0.001,0.01],
         'mlpclassifier__solver':['lbfgs',  'adam']
        }

grid_search_nn = GridSearchCV(pipe, param_grid = params,cv = KFold(n_splits=5, shuffle=True),n_jobs=-1)

In [111]:
train_outcome_small.state = train_outcome_small.state.astype(int)

In [117]:
grid_search_nn.fit(train_features_small,train_outcome_small)

GridSearchCV(cv=KFold(n_splits=5, random_state=None, shuffle=True),
       error_score='raise-deprecating',
       estimator=Pipeline(memory=None,
     steps=[('variancethreshold', VarianceThreshold(threshold=0.1)), ('selectkbest', SelectKBest(k=10, score_func=<function f_classif at 0x1272c8d08>)), ('mlpclassifier', MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08...=True, solver='adam', tol=0.0001,
       validation_fraction=0.1, verbose=False, warm_start=False))]),
       fit_params=None, iid='warn', n_jobs=-1,
       param_grid={'mlpclassifier__alpha': [0.0001, 0.001, 0.01], 'mlpclassifier__solver': ['lbfgs', 'adam']},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [118]:
grid_search_nn.score(validation_features,validation_outcome)

0.6422518723477845